### Simple Conversion from an PDF to it's markdown representation.

Extract the full text via PyMuPDF, convert the page to an image and give both to
a Vision Model and recreate the PDF Structure in markdown format.

Work's really well with a fine tuned model and text only PDF's.

In [ ]:
from pathlib import Path

import fitz
import os
import ollama

In [21]:
pages = fitz.open("<pathToPDF")
pages[0].get_pixmap().save("page.png")
full_text = pages[0].get_text()

In [43]:
query = """Extrahiere den vollständigen Text aus dem Bild und formatiere ihn in Markdown, indem du den mit PyMuPdf extrahierten Text als Referenz verwendest. Stelle sicher, dass die Markdown-Struktur korrekt wiedergegeben wird, mit besonderem Fokus auf Überschriften und Listen:

- Gib den Text eins zu eins wieder, ohne Informationen zu verändern oder hinzuzufügen.
- Verwende nur einfache `#`-Überschriften für fettgedruckte rote Texte, alles andere soll as normaler Text erscheinen.
- Behalte alle Aufzählungen, nummerierten Listen und Unterlisten im Markdown-Format exakt bei.
- Das Bild kann mit normalem Text beginnen und muss nicht mit einer Überschrift beginnen.

**PyMuPdf Text:**
```
{full_text}
```""".format(full_text=full_text)

In [ ]:
response = ollama.chat(
    model="llama3.2-vision:latest",
    stream=False,
    messages=[
        {
            "role": "user",
            "content": query,
            "images": [Path(os.getcwd(), "page.png")],
            "format": "markdown",
        },
    ],
)
print(response["message"]["content"])